#### Capstone Project Week 3 Note1(Part 1)

## ASSIGNMENT DESCRIPTION AND REQUIREMENTS

For this assignment, you will be required to explore and cluster the neighborhoods in Toronto.

Start by creating a new Notebook for this assignment.
Use the Notebook to build the code to scrape the following Wikipedia page, https://en.wikipedia.org/wiki/List_of_postal_codes_of_Canada:_M, in order to obtain the data that is in the table of postal codes and to transform the data into a pandas dataframe like the one shown below:

In [2]:
# importing the necessary libraries for this project as shown below

import pandas as pd
from bs4 import BeautifulSoup
import requests
import numpy as np
from geopy.geocoders import Nominatim # convert an address into latitude and longitude values
from pandas.io.json import json_normalize  # tranform JSON file into a pandas dataframe

import folium # map rendering library

# import k-means from clustering stage
from sklearn.cluster import KMeans

# Matplotlib and associated plotting modules
import matplotlib.cm as cm
import matplotlib.colors as colors

### Scrapping the Contents of the wikipedia page containing the table using BeautifulSoup as shown below

In [3]:
source = requests.get("https://en.wikipedia.org/wiki/List_of_postal_codes_of_Canada:_M").text #wikipedia page to be scrapped
soup = BeautifulSoup(source, 'lxml') #loading the entire wikipedia page for the desired data
table = soup.find("table") #getting the table portion of the loaded wikipedia page 
 



In [4]:
table_rows = table.tbody.find_all("tr") #getting all the rows from the table

# now, I will iterate through the whole table I have scrapped as shown below

res = []
for tr in table_rows:
    td = tr.find_all("td")
    row = [l.text for l in td]
    
    # Only process the cells that have an assigned borough. Ignore cells with a borough that is Not assigned.
    if row != [] and row[1] != "Not assigned" and row[1] != "Not assigned\n":
        # If a cell has a borough but a "Not assigned" neighborhood, then the neighborhood will be the same as the borough.
        if "Not assigned" in row[2]: 
            row[2] = row[1]
        res.append(row)

# Dataframe with 3 columns
pd.set_option('display.max_rows',None)
df = pd.DataFrame(res, columns = ["PostalCode", "Borough", "Neighborhood"])
df.head()

,PostalCode,Borough,Neighborhood
0,M3A\n,North York\n,Parkwoods\n
1,M4A\n,North York\n,Victoria Village\n
2,M5A\n,Downtown Toronto\n,"Regent Park, Harbourfront\n"
3,M6A\n,North York\n,"Lawrence Manor, Lawrence Heights\n"
4,M7A\n,Downtown Toronto\n,"Queen's Park, Ontario Provincial Government\n"


#### removing the trailing space('\n') from the above data frame using the following codes

In [5]:

df["PostalCode"] = df["PostalCode"].str.replace("\n","")
df["Borough"] = df["Borough"].str.replace("\n","")
df["Neighborhood"] = df["Neighborhood"].str.replace("\n","")
df.head()

,PostalCode,Borough,Neighborhood
0,M3A,North York,Parkwoods
1,M4A,North York,Victoria Village
2,M5A,Downtown Toronto,"Regent Park, Harbourfront"
3,M6A,North York,"Lawrence Manor, Lawrence Heights"
4,M7A,Downtown Toronto,"Queen's Park, Ontario Provincial Government"


### All the neighbourhoods with the same postal codes should be grouped together as shown below

In [6]:
df = df.groupby(["PostalCode", "Borough"])["Neighborhood"].apply(", ".join).reset_index()
df

,PostalCode,Borough,Neighborhood
0,M1B,Scarborough,"Malvern, Rouge"
1,M1C,Scarborough,"Rouge Hill, Port Union, Highland Creek"
2,M1E,Scarborough,"Guildwood, Morningside, West Hill"
3,M1G,Scarborough,Woburn
4,M1H,Scarborough,Cedarbrae
5,M1J,Scarborough,Scarborough Village
6,M1K,Scarborough,"Kennedy Park, Ionview, East Birchmount Park"
7,M1L,Scarborough,"Golden Mile, Clairlea, Oakridge"
8,M1M,Scarborough,"Cliffside, Cliffcrest, Scarborough Village West"
9,M1N,Scarborough,"Birch Cliff, Cliffside West"


### printing the number of rows of the dataframe.

In [7]:
print("The number of rows of the dataframe is: ", df.shape[0])

The number of rows of the dataframe is:  103


### ...........End of Notebook 1(PART 1)......................................

## .............................Notebook 2(Part 2) Begins Here.............................................................

## Question 2

Given that this package can be very unreliable, in case you are not able to get the geographical coordinates of the neighborhoods using the Geocoder package, here is a link to a csv file that has the geographical coordinates of each postal code: http://cocl.us/Geospatial_data

Use the Geocoder package or the csv file to create the following dataframe:



### 3. Get the latitude and the longitude coordinates of each neighborhood.

#### Getting the geographical coordinates of the neighbourhoods using the csv file provided via link in the assignment instead of using the geocoder package as shown below. I make use of pandas to read the provided csv file which I have downloaded from the link given in the assignment.

In [8]:
import pandas as pd
geo_df = pd.read_csv("Geospatial_Coordinates.csv")
pd.set_option("display.max_rows",None)
geo_df

,Postal Code,Latitude,Longitude
0,M1B,43.806686,-79.194353
1,M1C,43.784535,-79.160497
2,M1E,43.763573,-79.188711
3,M1G,43.770992,-79.216917
4,M1H,43.773136,-79.239476
5,M1J,43.744734,-79.239476
6,M1K,43.727929,-79.262029
7,M1L,43.711112,-79.284577
8,M1M,43.716316,-79.239476
9,M1N,43.692657,-79.264848


#### Now, I will merge two of the dataframes i.e the df and geo_df as a one dataframe as shown below. I want to retain all the information in the df dataframe and only part of the information in the geo_df dataframe that matches a given condition using a left join

In [9]:
toronto_complete_df = pd.merge(df,geo_df,how='left',left_on ='PostalCode',right_on = 'Postal Code')
toronto_complete_df.head()

,PostalCode,Borough,Neighborhood,Postal Code,Latitude,Longitude
0,M1B,Scarborough,"Malvern, Rouge",M1B,43.806686,-79.194353
1,M1C,Scarborough,"Rouge Hill, Port Union, Highland Creek",M1C,43.784535,-79.160497
2,M1E,Scarborough,"Guildwood, Morningside, West Hill",M1E,43.763573,-79.188711
3,M1G,Scarborough,Woburn,M1G,43.770992,-79.216917
4,M1H,Scarborough,Cedarbrae,M1H,43.773136,-79.239476


In [10]:
# The column 'Postal Code need to be dropped completely from the above dataframe as shown below'

toronto_complete_df.drop('Postal Code',axis=1,inplace=True)
toronto_complete_df

,PostalCode,Borough,Neighborhood,Latitude,Longitude
0,M1B,Scarborough,"Malvern, Rouge",43.806686,-79.194353
1,M1C,Scarborough,"Rouge Hill, Port Union, Highland Creek",43.784535,-79.160497
2,M1E,Scarborough,"Guildwood, Morningside, West Hill",43.763573,-79.188711
3,M1G,Scarborough,Woburn,43.770992,-79.216917
4,M1H,Scarborough,Cedarbrae,43.773136,-79.239476
5,M1J,Scarborough,Scarborough Village,43.744734,-79.239476
6,M1K,Scarborough,"Kennedy Park, Ionview, East Birchmount Park",43.727929,-79.262029
7,M1L,Scarborough,"Golden Mile, Clairlea, Oakridge",43.711112,-79.284577
8,M1M,Scarborough,"Cliffside, Cliffcrest, Scarborough Village West",43.716316,-79.239476
9,M1N,Scarborough,"Birch Cliff, Cliffside West",43.692657,-79.264848


#### .............................................End of Notebook 2(Part 2)...............................................................................................

## .....................................................Notebook 3(Part 3) Begins .................................................

## Question 3

Explore and cluster the neighborhoods in Toronto. You can decide to work with only boroughs that contain the word Toronto and then replicate the same analysis we did to the New York City data. It is up to you.

Just make sure:

1. to add enough Markdown cells to explain what you decided to do and to report any observations you make.
2. to generate maps to visualize your neighborhoods and how they cluster together.

### Exploring and Clustering the neighbourhoods in Toronto as shown below

#### Use geopy library to get the latitude and longitude values of New York City.

In order to define an instance of the geocoder, we need to define a user_agent. We will name our agent <em>tr_explorer</em>, as shown below.

In [11]:
address = 'Toronto, ON'

geolocator = Nominatim(user_agent="ny_explorer")
location = geolocator.geocode(address)
latitude = location.latitude
longitude = location.longitude
print('The geograpical coordinate of Toronto City are {}, {}.'.format(latitude, longitude))

The geograpical coordinate of Toronto City are 43.6534817, -79.3839347.


#### Create a map of Toronto City with neighborhoods superimposed on top.

In [12]:
# first, I will create map of Toronto using its latitude and longitude values before adding marker as shown below
toronto_map = folium.Map(location=[latitude, longitude], zoom_start=10)
toronto_map

### adding markers to the Torontan map above as shown below

In [13]:
for lat, lng, borough, neighborhood in zip(
        toronto_complete_df['Latitude'], 
        toronto_complete_df['Longitude'], 
        toronto_complete_df['Borough'], 
        toronto_complete_df['Neighborhood']):
    label = '{}, {}'.format(neighborhood, borough)
    label = folium.Popup(label, parse_html=True)
    folium.CircleMarker(
        [lat, lng],
        radius=5,
        popup=label,
        color='blue',
        fill=True,
        fill_color='#3186cc',
        fill_opacity=0.7,
        parse_html=False).add_to(toronto_map)  

toronto_map

#### MAPPING PART OF THE CITY  OF TORONTO

#### I decide to work with only boroughs that contain the word Toronto

In [14]:
# Now, getting the bourough which contain the word Toronto
df_only_toronto = toronto_complete_df[toronto_complete_df['Borough'].str.contains("Toronto")].reset_index(drop=True)
df_only_toronto.head()

,PostalCode,Borough,Neighborhood,Latitude,Longitude
0,M4E,East Toronto,The Beaches,43.676357,-79.293031
1,M4K,East Toronto,"The Danforth West, Riverdale",43.679557,-79.352188
2,M4L,East Toronto,"India Bazaar, The Beaches West",43.668999,-79.315572
3,M4M,East Toronto,Studio District,43.659526,-79.340923
4,M4N,Central Toronto,Lawrence Park,43.728020,-79.388790


### Lets see the map for only the borough containing Toronto as shown below

In [15]:
map_only_toronto = folium.Map(location=[latitude, longitude], zoom_start=12)
for lat, lng, borough, neighborhood in zip(
        df_only_toronto['Latitude'], 
        df_only_toronto['Longitude'], 
        df_only_toronto['Borough'], 
        df_only_toronto['Neighborhood']):
    label = '{}, {}'.format(neighborhood, borough)
    label = folium.Popup(label, parse_html=True)
    folium.CircleMarker(
        [lat, lng],
        radius=5,
        popup=label,
        color='blue',
        fill=True,
        fill_color='#3186cc',
        fill_opacity=0.7,
        parse_html=False).add_to(map_only_toronto)  

map_only_toronto

#### Define Foursquare Credentials and Version

In [16]:
CLIENT_ID = '' # my Foursquare ID which I used but cleared it before posting for privacy purpose
CLIENT_SECRET = '' # my Foursquare Secret
VERSION = '20180605' # Foursquare API version


### Exploring the first neighbourhood in the dataframe, df_only_toronto as shown below

In [17]:
first_neighborhood_name = df_only_toronto.loc[0, 'Neighborhood']
print(f"The first neighborhood name is '{first_neighborhood_name}'.")

The first neighborhood name is 'The Beaches'.


##### Getting the neighborhood's latitude and longitude values.

In [18]:
neighborhood_latitude = df_only_toronto.loc[0, 'Latitude'] # neighborhood latitude value
neighborhood_longitude = df_only_toronto.loc[0, 'Longitude'] # neighborhood longitude value

neighborhood_name = df_only_toronto.loc[0, 'Neighborhood'] # neighborhood name

print('Latitude and longitude values of {} are {}, {}.'.format(neighborhood_name, 
                                                               neighborhood_latitude, 
                                                               neighborhood_longitude))

Latitude and longitude values of The Beaches are 43.67635739999999, -79.2930312.


#### Now, let's get the top 100 venues that are in The Beaches within a radius of 500 meters.

First, let's create the GET request URL. Name your URL **url**.

In [19]:
radius = 500
LIMIT = 100

url = 'https://api.foursquare.com/v2/venues/explore?&client_id={}&client_secret={}&v={}&ll={},{}&radius={}&limit={}'.format(
    CLIENT_ID, 
    CLIENT_SECRET, 
    VERSION, 
    neighborhood_latitude, 
    neighborhood_longitude, 
    radius, 
    LIMIT)


Sending the GET request and examining the results

In [20]:
results = requests.get(url).json()


#### Writing a function that extracts the categories of the data

In [21]:
def get_category_type(row):
    try:
        categories_list = row['categories']
    except:
        categories_list = row['venue.categories']
        
    if len(categories_list) == 0:
        return None
    else:
        return categories_list[0]['name']

Now, cleaning the json and structure it into a *pandas* dataframe.

In [22]:
venues = results['response']['groups'][0]['items']
nearby_venues = json_normalize(venues) # flatten JSON
# filter columns
filtered_columns = ['venue.name', 'venue.categories', 'venue.location.lat', 'venue.location.lng']
nearby_venues =nearby_venues.loc[:, filtered_columns]
# filter the category for each row
nearby_venues['venue.categories'] = nearby_venues.apply(get_category_type, axis=1)

# clean columns
nearby_venues.columns = [col.split(".")[-1] for col in nearby_venues.columns]

nearby_venues


,name,categories,lat,lng
0,Glen Manor Ravine,Trail,43.676821,-79.293942
1,The Big Carrot Natural Food Market,Health Food Store,43.678879,-79.297734
2,Grover Pub and Grub,Pub,43.679181,-79.297215
3,Upper Beaches,Neighborhood,43.680563,-79.292869


And how many venues were returned by Foursquare?

In [23]:
print('{} venues were returned by Foursquare.'.format(nearby_venues.shape[0]))

4 venues were returned by Foursquare.


##  Explore Neighborhoods in Toronto

#### Let's create a function to repeat the same process to all the neighborhoods in Toronto

In [24]:
def getNearbyVenues(names, latitudes, longitudes, radius=500):
    venues_list=[]
    
    for name, lat, lng in zip(names, latitudes, longitudes):
        # print(name)
            
        # create the API request URL
        url = 'https://api.foursquare.com/v2/venues/explore?&client_id={}&client_secret={}&v={}&ll={},{}&radius={}&limit={}'.format(
            CLIENT_ID, 
            CLIENT_SECRET, 
            VERSION, 
            lat, 
            lng, 
            radius, 
            LIMIT)
            
        # make the GET request
        results = requests.get(url).json()["response"]['groups'][0]['items']
        
        # return only relevant information for each nearby venue
        venues_list.append([(
            name, 
            lat, 
            lng, 
            v['venue']['name'], 
            v['venue']['location']['lat'], 
            v['venue']['location']['lng'],  
            v['venue']['categories'][0]['name']) for v in results])

    nearby_venues = pd.DataFrame([item for venue_list in venues_list for item in venue_list])
    nearby_venues.columns = ['Neighborhood', 
                  'Neighborhood Latitude', 
                  'Neighborhood Longitude', 
                  'Venue', 
                  'Venue Latitude', 
                  'Venue Longitude', 
                  'Venue Category']
    
    return(nearby_venues)

#### Now write the code to run the above function on each neighborhood and create a new dataframe called *df_only_toronto_venue*.

In [25]:
df_only_toronto_venue = getNearbyVenues(names=df_only_toronto['Neighborhood'],
                                   latitudes=df_only_toronto['Latitude'],
                                   longitudes=df_only_toronto['Longitude']
                                  )
df_only_toronto_venue.head()

,Neighborhood,Neighborhood Latitude,Neighborhood Longitude,Venue,Venue Latitude,Venue Longitude,Venue Category
0,The Beaches,43.676357,-79.293031,Glen Manor Ravine,43.676821,-79.293942,Trail
1,The Beaches,43.676357,-79.293031,The Big Carrot Natural Food Market,43.678879,-79.297734,Health Food Store
2,The Beaches,43.676357,-79.293031,Grover Pub and Grub,43.679181,-79.297215,Pub
3,The Beaches,43.676357,-79.293031,Upper Beaches,43.680563,-79.292869,Neighborhood
4,"The Danforth West, Riverdale",43.679557,-79.352188,MenEssentials,43.677820,-79.351265,Cosmetics Shop


#### Let's check the size of the resulting dataframe

In [37]:
print(df_only_toronto_venue.shape)

(1613, 7)


Let's check how many venues were returned for each neighborhood

In [26]:
df_only_toronto_venue.groupby('Neighborhood').count()

,Neighborhood Latitude,Neighborhood Longitude,Venue,Venue Latitude,Venue Longitude,Venue Category
Neighborhood,,,,,,
Berczy Park,54,54,54,54,54,54
"Brockton, Parkdale Village, Exhibition Place",23,23,23,23,23,23
Business reply mail Processing Centre,17,17,17,17,17,17
"CN Tower, King and Spadina, Railway Lands, Harbourfront West, Bathurst Quay, South Niagara, Island airport",15,15,15,15,15,15
Central Bay Street,65,65,65,65,65,65
Christie,17,17,17,17,17,17
Church and Wellesley,77,77,77,77,77,77
"Commerce Court, Victoria Hotel",100,100,100,100,100,100
Davisville,38,38,38,38,38,38


#### Let's find out how many unique categories can be curated from all the returned venues

In [27]:
print('There are {} uniques categories.'.format(len(df_only_toronto_venue['Venue Category'].unique())))

There are 239 uniques categories.


## 3. Analyze Each Neighborhood

In [28]:
# one hot encoding
df_only_toronto_venue_onehot = pd.get_dummies(df_only_toronto_venue[['Venue Category']], prefix="", prefix_sep="")

# add neighborhood column back to dataframe
df_only_toronto_venue_onehot['Neighborhood'] = df_only_toronto_venue['Neighborhood'] 

# move neighborhood column to the first column
fixed_columns = [df_only_toronto_venue_onehot.columns[-1]] + list(df_only_toronto_venue_onehot.columns[:-1])
df_only_toronto_venue_onehot = df_only_toronto_venue_onehot[fixed_columns]

df_only_toronto_venue_onehot.head()


,Yoga Studio,Afghan Restaurant,Airport,Airport Food Court,Airport Gate,Airport Lounge,Airport Service,Airport Terminal,American Restaurant,Antique Shop,...,Theme Restaurant,Toy / Game Store,Trail,Train Station,Vegetarian / Vegan Restaurant,Video Game Store,Vietnamese Restaurant,Wine Bar,Wings Joint,Women's Store
0,0,0,0,0,0,0,0,0,0,0,...,0,0,1,0,0,0,0,0,0,0
1,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
2,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
3,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
4,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0


#### Next, let's group rows by neighborhood and by taking the mean of the frequency of occurrence of each category

In [30]:
df_only_toronto_venue_onehot_grouped = df_only_toronto_venue_onehot.groupby('Neighborhood').mean().reset_index()
df_only_toronto_venue_onehot_grouped.head()

,Neighborhood,Yoga Studio,Afghan Restaurant,Airport,Airport Food Court,Airport Gate,Airport Lounge,Airport Service,Airport Terminal,American Restaurant,...,Theme Restaurant,Toy / Game Store,Trail,Train Station,Vegetarian / Vegan Restaurant,Video Game Store,Vietnamese Restaurant,Wine Bar,Wings Joint,Women's Store
0,Berczy Park,0.000000,0.0,0.000000,0.000000,0.000000,0.000000,0.0,0.000000,0.0,...,0.0,0.0,0.0,0.0,0.018519,0.0,0.0,0.0,0.0,0.0
1,"Brockton, Parkdale Village, Exhibition Place",0.000000,0.0,0.000000,0.000000,0.000000,0.000000,0.0,0.000000,0.0,...,0.0,0.0,0.0,0.0,0.000000,0.0,0.0,0.0,0.0,0.0
2,Business reply mail Processing Centre,0.000000,0.0,0.000000,0.000000,0.000000,0.000000,0.0,0.000000,0.0,...,0.0,0.0,0.0,0.0,0.000000,0.0,0.0,0.0,0.0,0.0
3,"CN Tower, King and Spadina, Railway Lands, Har...",0.000000,0.0,0.066667,0.066667,0.066667,0.066667,0.2,0.133333,0.0,...,0.0,0.0,0.0,0.0,0.000000,0.0,0.0,0.0,0.0,0.0
4,Central Bay Street,0.015385,0.0,0.000000,0.000000,0.000000,0.000000,0.0,0.000000,0.0,...,0.0,0.0,0.0,0.0,0.015385,0.0,0.0,0.0,0.0,0.0


#### Let's confirm the new size

In [31]:
df_only_toronto_venue_onehot_grouped.shape

(39, 239)

#### Check the 10 most common venues in each neighborhood.

In [32]:
def return_most_common_venues(row, num_top_venues):
    row_categories = row.iloc[1:]
    row_categories_sorted = row_categories.sort_values(ascending=False)
    return row_categories_sorted.index.values[0:num_top_venues]

num_top_venues = 10

indicators = ['st', 'nd', 'rd']

# create columns according to number of top venues
columns = ['Neighborhood']
for ind in np.arange(num_top_venues):
    try:
        columns.append('{}{} Most Common Venue'.format(ind+1, indicators[ind]))
    except:
        columns.append('{}th Most Common Venue'.format(ind+1))

# create a new dataframe
neighborhoods_venues_sorted = pd.DataFrame(columns=columns)
neighborhoods_venues_sorted['Neighborhood'] = df_only_toronto_venue_onehot_grouped['Neighborhood']

for ind in np.arange(df_only_toronto_venue_onehot_grouped.shape[0]):
    neighborhoods_venues_sorted.iloc[ind, 1:] = return_most_common_venues(df_only_toronto_venue_onehot_grouped.iloc[ind, :], num_top_venues)

neighborhoods_venues_sorted.head()

,Neighborhood,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue,5th Most Common Venue,6th Most Common Venue,7th Most Common Venue,8th Most Common Venue,9th Most Common Venue,10th Most Common Venue
0,Berczy Park,Coffee Shop,Cocktail Bar,Beer Bar,Seafood Restaurant,Restaurant,Bakery,Café,Cheese Shop,Grocery Store,Bistro
1,"Brockton, Parkdale Village, Exhibition Place",Café,Breakfast Spot,Nightclub,Coffee Shop,Grocery Store,Stadium,Intersection,Bakery,Italian Restaurant,Restaurant
2,Business reply mail Processing Centre,Smoke Shop,Garden,Restaurant,Auto Workshop,Fast Food Restaurant,Farmers Market,Burrito Place,Recording Studio,Brewery,Butcher
3,"CN Tower, King and Spadina, Railway Lands, Har...",Airport Service,Airport Terminal,Sculpture Garden,Rental Car Location,Plane,Boat or Ferry,Boutique,Harbor / Marina,Airport Lounge,Airport Gate
4,Central Bay Street,Coffee Shop,Italian Restaurant,Sandwich Place,Café,Burger Joint,Japanese Restaurant,Ice Cream Shop,Bar,Thai Restaurant,Salad Place


## 4. Cluster Neighborhoods

Run *k*-means to cluster the neighborhood into 5 clusters.

In [33]:
# set number of clusters
kclusters = 5

df_only_toronto_venue_onehot_grouped_clustering = df_only_toronto_venue_onehot_grouped.drop('Neighborhood', 1)

# run k-means clustering
kmeans = KMeans(n_clusters=kclusters, random_state=0).fit(df_only_toronto_venue_onehot_grouped_clustering)

# check cluster labels generated for each row in the dataframe
kmeans.labels_[0:10] 

array([1, 1, 1, 1, 1, 1, 1, 1, 1, 1])

Let's create a new dataframe that includes the cluster as well as the top 10 venues for each neighborhood.

In [34]:
# add clustering labels
neighborhoods_venues_sorted.insert(0, 'Cluster Labels', kmeans.labels_)

df_only_toronto_merged = df_only_toronto

# merge toronto_grouped with toronto_data to add latitude/longitude for each neighborhood
df_only_toronto_merged = df_only_toronto_merged.join(neighborhoods_venues_sorted.set_index('Neighborhood'), on='Neighborhood')

df_only_toronto_merged.head() # check the last columns!

,PostalCode,Borough,Neighborhood,Latitude,Longitude,Cluster Labels,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue,5th Most Common Venue,6th Most Common Venue,7th Most Common Venue,8th Most Common Venue,9th Most Common Venue,10th Most Common Venue
0,M4E,East Toronto,The Beaches,43.676357,-79.293031,0,Pub,Health Food Store,Trail,Women's Store,Discount Store,Deli / Bodega,Department Store,Dessert Shop,Diner,Dog Run
1,M4K,East Toronto,"The Danforth West, Riverdale",43.679557,-79.352188,1,Greek Restaurant,Italian Restaurant,Coffee Shop,Restaurant,Ice Cream Shop,Furniture / Home Store,Fruit & Vegetable Store,Pub,Pizza Place,Lounge
2,M4L,East Toronto,"India Bazaar, The Beaches West",43.668999,-79.315572,1,Fast Food Restaurant,Pet Store,Pub,Brewery,Sandwich Place,Burrito Place,Italian Restaurant,Restaurant,Fish & Chips Shop,Ice Cream Shop
3,M4M,East Toronto,Studio District,43.659526,-79.340923,1,Café,Coffee Shop,Bakery,Gastropub,Brewery,American Restaurant,Comfort Food Restaurant,Bookstore,Seafood Restaurant,Sandwich Place
4,M4N,Central Toronto,Lawrence Park,43.728020,-79.388790,3,Park,Bus Line,Swim School,Women's Store,Deli / Bodega,Ethiopian Restaurant,Electronics Store,Eastern European Restaurant,Donut Shop,Doner Restaurant


#### Finally, let's visualize the resulting clusters

In [35]:
# create map
map_clusters = folium.Map(location=[latitude, longitude], zoom_start=11)

# set color scheme for the clusters
x = np.arange(kclusters)
ys = [i + x + (i*x)**2 for i in range(kclusters)]
colors_array = cm.rainbow(np.linspace(0, 1, len(ys)))
rainbow = [colors.rgb2hex(i) for i in colors_array]

# add markers to the map
markers_colors = []
for lat, lon, poi, cluster in zip(
        df_only_toronto_merged['Latitude'], 
        df_only_toronto_merged['Longitude'], 
        df_only_toronto_merged['Neighborhood'], 
        df_only_toronto_merged['Cluster Labels']):
    label = folium.Popup(str(poi) + ' Cluster ' + str(cluster), parse_html=True)
    folium.CircleMarker(
        [lat, lon],
        radius=5,
        popup=label,
        color=rainbow[cluster-1],
        fill=True,
        fill_color=rainbow[cluster-1],
        fill_opacity=0.7).add_to(map_clusters)
       
map_clusters

## Examine Clusters

Now, you can examine each cluster and determine the discriminating venue categories that distinguish each cluster. Based on the defining categories, you can then assign a name to each cluster. I will leave this exercise to you.

#### Cluster 1

In [36]:
df_only_toronto_merged.loc[df_only_toronto_merged['Cluster Labels'] == 0, df_only_toronto_merged.columns[[1] + list(range(5, df_only_toronto_merged.shape[1]))]]


,Borough,Cluster Labels,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue,5th Most Common Venue,6th Most Common Venue,7th Most Common Venue,8th Most Common Venue,9th Most Common Venue,10th Most Common Venue
0,East Toronto,0,Pub,Health Food Store,Trail,Women's Store,Discount Store,Deli / Bodega,Department Store,Dessert Shop,Diner,Dog Run


#### Cluster 2

In [37]:
df_only_toronto_merged.loc[df_only_toronto_merged['Cluster Labels'] == 1, df_only_toronto_merged.columns[[1] + list(range(5, df_only_toronto_merged.shape[1]))]]

,Borough,Cluster Labels,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue,5th Most Common Venue,6th Most Common Venue,7th Most Common Venue,8th Most Common Venue,9th Most Common Venue,10th Most Common Venue
1,East Toronto,1,Greek Restaurant,Italian Restaurant,Coffee Shop,Restaurant,Ice Cream Shop,Furniture / Home Store,Fruit & Vegetable Store,Pub,Pizza Place,Lounge
2,East Toronto,1,Fast Food Restaurant,Pet Store,Pub,Brewery,Sandwich Place,Burrito Place,Italian Restaurant,Restaurant,Fish & Chips Shop,Ice Cream Shop
3,East Toronto,1,Café,Coffee Shop,Bakery,Gastropub,Brewery,American Restaurant,Comfort Food Restaurant,Bookstore,Seafood Restaurant,Sandwich Place
5,Central Toronto,1,Dance Studio,Sandwich Place,Department Store,Park,Breakfast Spot,Hotel,Food & Drink Shop,Dessert Shop,Diner,Women's Store
6,Central Toronto,1,Clothing Store,Coffee Shop,Yoga Studio,Sporting Goods Shop,Salon / Barbershop,Restaurant,Rental Car Location,Café,Chinese Restaurant,Park
7,Central Toronto,1,Pizza Place,Sandwich Place,Dessert Shop,Coffee Shop,Italian Restaurant,Gym,Thai Restaurant,Sushi Restaurant,Café,Diner
9,Central Toronto,1,Pub,Coffee Shop,American Restaurant,Supermarket,Sushi Restaurant,Bank,Sports Bar,Fried Chicken Joint,Restaurant,Bagel Shop
11,Downtown Toronto,1,Coffee Shop,Restaurant,Italian Restaurant,Park,Chinese Restaurant,Bakery,Pizza Place,Café,Pub,Butcher
12,Downtown Toronto,1,Coffee Shop,Japanese Restaurant,Sushi Restaurant,Restaurant,Gay Bar,Café,Pub,Men's Store,Mediterranean Restaurant,Hotel
13,Downtown Toronto,1,Coffee Shop,Bakery,Pub,Park,Breakfast Spot,Café,Theater,Mexican Restaurant,Shoe Store,Restaurant


#### Cluster 3

In [38]:
df_only_toronto_merged.loc[df_only_toronto_merged['Cluster Labels'] == 2, df_only_toronto_merged.columns[[1] + list(range(5, df_only_toronto_merged.shape[1]))]]

,Borough,Cluster Labels,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue,5th Most Common Venue,6th Most Common Venue,7th Most Common Venue,8th Most Common Venue,9th Most Common Venue,10th Most Common Venue
8,Central Toronto,2,Restaurant,Park,Trail,Discount Store,Dance Studio,Deli / Bodega,Department Store,Dessert Shop,Diner,Women's Store
10,Downtown Toronto,2,Park,Playground,Trail,Women's Store,Distribution Center,Deli / Bodega,Department Store,Dessert Shop,Diner,Discount Store
23,Central Toronto,2,Park,Sushi Restaurant,Jewelry Store,Trail,Women's Store,Dog Run,Dessert Shop,Diner,Discount Store,Distribution Center


#### Cluster 4

In [39]:
df_only_toronto_merged.loc[df_only_toronto_merged['Cluster Labels'] == 3, df_only_toronto_merged.columns[[1] + list(range(5, df_only_toronto_merged.shape[1]))]]


,Borough,Cluster Labels,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue,5th Most Common Venue,6th Most Common Venue,7th Most Common Venue,8th Most Common Venue,9th Most Common Venue,10th Most Common Venue
4,Central Toronto,3,Park,Bus Line,Swim School,Women's Store,Deli / Bodega,Ethiopian Restaurant,Electronics Store,Eastern European Restaurant,Donut Shop,Doner Restaurant


#### Cluster 5

In [40]:
df_only_toronto_merged.loc[df_only_toronto_merged['Cluster Labels'] == 4, df_only_toronto_merged.columns[[1] + list(range(5, df_only_toronto_merged.shape[1]))]]


,Borough,Cluster Labels,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue,5th Most Common Venue,6th Most Common Venue,7th Most Common Venue,8th Most Common Venue,9th Most Common Venue,10th Most Common Venue
22,Central Toronto,4,Home Service,Garden,Women's Store,Dance Studio,Ethiopian Restaurant,Electronics Store,Eastern European Restaurant,Donut Shop,Doner Restaurant,Dog Run
